<a href="https://colab.research.google.com/github/21cs076/21cs076/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov11")

In [3]:
import albumentations as A
import cv2
import os
import numpy as np

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=40, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

def read_label(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            if line.strip():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])
    return np.array(labels)

def save_label(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(map(str, label)) + '\n')

def augment_and_save(image_path, label_path, save_image_path, save_label_path, transform, num_augmented):
    image = cv2.imread(image_path)
    labels = read_label(label_path)

    if labels.size == 0:  # Skip augmentation if no labels are found
        return

    class_labels = labels[:, 0]
    bboxes = labels[:, 1:]

    for i in range(num_augmented):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = transformed['image']
        augmented_bboxes = transformed['bboxes']

        if len(augmented_bboxes) == 0:  # Skip if no bounding boxes remain after augmentation
            continue

        augmented_labels = np.hstack((np.array(transformed['class_labels'])[:, np.newaxis], augmented_bboxes))

        augmented_image_path = os.path.join(save_image_path, f"aug_{i}_{os.path.basename(image_path)}")
        augmented_label_path = os.path.join(save_label_path, f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(augmented_image_path, augmented_image)
        save_label(augmented_label_path, augmented_labels)

# Paths to your dataset images and labels
dataset_image_path = f"{dataset.location}/train/images"
dataset_label_path = f"{dataset.location}/train/labels"
augmented_image_path = f"{dataset.location}/aug/images"
augmented_label_path = f"{dataset.location}/aug/labels"

os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Augment each image and its corresponding label in the dataset
for image_name in os.listdir(dataset_image_path):
    image_path = os.path.join(dataset_image_path, image_name)
    label_path = os.path.join(dataset_label_path, image_name.replace('.jpg', '.txt'))
    augment_and_save(image_path, label_path, augmented_image_path, augmented_label_path, transform, num_augmented=5)

A new version of Albumentations is available: 2.0.0 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [4]:
import shutil
import os

# Ensure the augmented directories exist
os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Copy original images to the augmented images directory
for image_name in os.listdir(dataset_image_path):
    src_image_path = os.path.join(dataset_image_path, image_name)
    dst_image_path = os.path.join(augmented_image_path, image_name)
    shutil.copy(src_image_path, dst_image_path)

# Copy original labels to the augmented labels directory
for label_name in os.listdir(dataset_label_path):
    src_label_path = os.path.join(dataset_label_path, label_name)
    dst_label_path = os.path.join(augmented_label_path, label_name)
    shutil.copy(src_label_path, dst_label_path)


import yaml
data = {
    'train': f"{dataset.location}/aug/images",
    'val': f"{dataset.location}/valid/images",
    'test': f"{dataset.location}/test/images",
    'nc': 2,
    'names': ['Fissure', 'Landslide']
}
with open(f"{dataset.location}/data.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

In [5]:
from ultralytics import YOLO
from google.colab import files

# Load a pre-trained YOLOv8 model
model = YOLO("yolo11m.pt")

# Train the model on your Landslide dataset
model.train(
    data = f"{dataset.location}/data.yaml",  # Path to your data.yaml file
    epochs=50,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = f"{dataset.location}/runs/train",  # Directory to save training results
    name="landslide_detection",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)

# Download the saved model
files.download(f"{dataset.location}/runs/train/landslide_detection/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 38.8M/38.8M [00:00<00:00, 135MB/s]


Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/content/Landslide-Detection-Augmentation-12/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/Landslide-Detection-Augmentation-12/runs/train, name=landslide_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sa

100%|██████████| 755k/755k [00:00<00:00, 21.3MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  1   1380352  ultralytics.nn.modules.block.C3k2            [512, 512, 1, True]           
  7                  -1  1   2360320  ultralytics

...


50 epochs completed in 2.820 hours.
Optimizer stripped from /content/Landslide-Detection-Augmentation-12/runs/train/landslide_detection/weights/last.pt, 40.5MB
Optimizer stripped from /content/Landslide-Detection-Augmentation-12/runs/train/landslide_detection/weights/best.pt, 40.5MB

Validating /content/Landslide-Detection-Augmentation-12/runs/train/landslide_detection/weights/best.pt...
Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


                   all        115        162      0.604      0.402       0.45      0.209
               Fissure         29         60      0.609       0.25       0.35      0.151
             Landslide         61        102      0.598      0.554       0.55      0.267
Speed: 0.1ms preprocess, 7.2ms inference, 0.0ms loss, 3.3ms postprocess per image
Results saved to /content/Landslide-Detection-Augmentation-12/runs/train/landslide_detection


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO(f"{dataset.location}/runs/train/landslide_detection/weights/best.pt")

# Evaluate the model using the val() method
metrics = model.val(data=f"{dataset.location}/data.yaml", split='test')

# Print evaluation metrics
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

Ultralytics 8.3.65 🚀 Python-3.11.11 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11m summary (fused): 303 layers, 20,031,574 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/test/labels... 60 images, 21 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 386.96it/s]

val: New cache created: /content/Landslide-Detection-Augmentation-12/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.06it/s]


                   all         60         97      0.522      0.508      0.455       0.18
               Fissure         19         30      0.607        0.4      0.424      0.176
             Landslide         30         67      0.438      0.616      0.485      0.185
Speed: 0.3ms preprocess, 32.3ms inference, 0.0ms loss, 5.2ms postprocess per image
Results saved to runs/detect/val
mAP50: 0.45466657024736723
mAP75: 0.10034620093964293


In [7]:
#from google.colab import files
#uploaded = files.upload()

#import os
#os.listdir('/content')

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')  # Set the locale to a UTF-8 compatible one

import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
install("onnx") # Installing onnx package

import torch
import onnx
import numpy as np
from google.colab import files

onnx_model_path = f"{dataset.location}/best.onnx"
torch.onnx.export(
    model,
    torch.randn(1, 3, 640, 640),  # Dummy input for the model
    onnx_model_path,
    opset_version=11,
    input_names=["input"],
    output_names=["output"]
)

files.download(f"{dataset.location}/best.onnx")

In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics